## Airflow on Kubernetes

This notebook can be used to launch a standard instance of Jupyterhub which includes a proxy server, and an instance of Jupyterhub. 

Notes on the Server: 
`/etc/jupyterhub/config` is where the configuration is for jupyterhub. 



In [ ]:
cf = dict()
cf['cloud_provider']='azure'      #azure of google
cf['project_name']='kuberlytics'  #Must be globally unique.
cf['restore_config']=False   #Only True for existing or previous cluster   
cf['cluster_name']='kuberlytics'         #Only use if restore_config==True
cf['config_file']='config/config.yaml'  #The configuration will be overwritten if retreived.
cf['path']='/home/jovyan/launch/'  #Path to admintools repository
cf['ssh_dir']='/home/jovyan/.ssh/'      #ssh directory

In [ ]:
import sys 
import importlib
import ruamel.yaml
sys.path.append(cf['path']+"lib/kuberutils") #Adds a local library path.
import kuberutils as ku   #This imports some utilities.
importlib.reload(ku)      #This just reloads the utilities.
cf=ku.initialize(cf)  #This initializes a configuration object.

### Print Configuration
Optionally you can print the configuration and common commands for your desired cluster. You can use this as a reference and copy and paste into the terminal.

In [ ]:
#This will print common commands for your clustl
print(ruamel.yaml.dump(cf, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))

In [ ]:
cf['air_config_new']=cf['path']+"config/airflow/airflow.new.yaml"
cf['air_config_old']=cf['path']+"config/airflow/airflow.old.yaml"

In [ ]:
ku.bash_command("mkdir "+cf['path']+"config/airflow")

In [ ]:
ku.bash_command("wget -O "+cf['air_config_new']+" "+cf['air_config_new_url'])
ku.bash_command("wget -O "+cf['air_config_old']+" "+cf['air_config_old_url'])

In [ ]:
ku.bash_command("kubectl create -f  "+cf['air_config_old'])

In [ ]:
#This will parse the multiple .yaml files. 
stream = open(cf['air_config_old'], "r")
docs = ruamel.yaml.load_all(stream)
for doc in docs:
    for k,v in doc.items():
        print (k, "->", v)
    print ("\n")

In [ ]:
print(ku.bash_command("kubectl get pods"))

In [ ]:
print(ku.bash_command("kubectl get services")) 

In [ ]:

print(ku.bash_command("kubectl port-forward web-2430772865-brf6p 8080:8080")) 

## Cleanup the Installation
This will cleanup the installation, deleting the instance of Jupyterhub.

In [ ]:
#Delete Jupyterhub with Helm.
kubectl delete -f airflow.all.yaml

In [ ]:
ku.bash_command("kubectl delete -f  "+cf['air_config_old'])

In [ ]:
#Delete Jupyterhub with Helm.
print(ku.bash_command('jup_delete_namespace',cf))